In [150]:
import os
import pandas as pd
import glob
import numpy as np
import re

## 1. Unemployment (annual averages by county 2000-2020)

In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/economic/unemployment'
os.chdir(path)
file_pattern = "*.xlsx"
xlsx_files = glob.glob(file_pattern)

merged_data = []

for file in xlsx_files:
    df = pd.read_excel(file)
    df = df.drop(index=df.index[:5])
    df = df.drop(index=df.index[-3:])
    df = df.drop('Unnamed: 5', axis=1)
    new_column_names = ['LAUScode', 'StateFIPS', 'CountyFIPS', 'CountyName', 'Year', 'LaborForce', 'Employed', 'Unemployed', 'UnemploymentRate']
    df = df.rename(columns=dict(zip(df.columns, new_column_names)))
    merged_data.append(df)

merged_df = pd.concat(merged_data, ignore_index=True)

In [ ]:
merged_df = merged_df.replace("N.A.", np.nan)

In [ ]:
columns_to_convert = ['LaborForce', 'Unemployed', 'Employed']

for column in columns_to_convert:
    merged_df[column] = pd.to_numeric(merged_df[column], errors='coerce')

In [ ]:
merged_df['Unemployment'] = merged_df['Unemployed'] / merged_df['LaborForce'] * 100

In [ ]:
merged_df['FIPS'] = merged_df['StateFIPS'] + merged_df['CountyFIPS']

In [ ]:
merged_df = merged_df[['Year','FIPS','Unemployment','LaborForce']]

In [ ]:
merged_df['FIPS'] = merged_df['FIPS'].astype(str)


In [ ]:
merged_df[merged_df['Unemployment'].isna()]['Year'].unique()

array(['2005', '2006', '2020'], dtype=object)

In [ ]:
merged_df = merged_df.sort_values(by="Year")
merged_df

,Year,FIPS,Unemployment,LaborForce
48264,2000,72137,7.370376,35005.0
48260,2000,72129,10.866957,13334.0
48259,2000,72127,7.379081,173897.0
48258,2000,72125,13.371197,12325.0
48257,2000,72123,13.828889,8287.0
...,...,...,...,...
67576,2020,72147,NaN,NaN
65970,2020,30025,3.920386,1658.0
67556,2020,72107,NaN,NaN
67568,2020,72131,NaN,NaN


In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs'
merged_df.to_csv(path + '/unemployment.csv', index=False)

## 2. Bankruptcy filings (2000-2020)

In [155]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/economic/bankruptcy filings/circuit and district court/'
os.chdir(path)
xlsx_files = glob.glob("*.xlsx")
xls_files = glob.glob('*.xls')

In [163]:
df = pd.read_excel(xls_files[0])

df = df.drop(index=df.index[:15])
df = df.dropna(subset=df.columns[1:], how='all')
df = df.drop(['Unnamed: 7','Unnamed: 8','Unnamed: 9', 'Unnamed: 10', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'], axis=1)
new_column_names = ['District', 'TotalFilings', 'Chapter7', 'Chapter11', 'Chapter12', 'Chapter13', 'Business', 'NonBusiness']
df = df.rename(columns=dict(zip(df.columns, new_column_names)))

df

WARNING *** file size (34961) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,District,TotalFilings,Chapter7,Chapter11,Chapter12,Chapter13,Business,NonBusiness
15,DC......,712,358,18,-,336,36,676
17,1ST...,29595,16930,282,27,12355,1193,28402
19,ME......,2304,1819,15,6,464,152,2152
20,MA......,13705,9061,143,4,4496,333,13372
21,NH......,2983,2100,20,3,860,327,2656
...,...,...,...,...,...,...,...,...
137,"FL,M....",26437,15825,229,8,10374,1109,25328
138,"FL,S....",12272,8419,272,2,3578,799,11473
139,"GA,N....",32320,15423,222,2,16673,1154,31166
140,"GA,M....",9446,3273,17,4,6152,161,9285


In [176]:
merged_data = []

for file in xls_files:
    df = pd.read_excel(file)
    df = df.drop(index=df.index[:15])
    df = df.dropna(subset=df.columns[1:], how='all')
    df = df.drop(['Unnamed: 7','Unnamed: 8','Unnamed: 9', 'Unnamed: 10', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'], axis=1)
    new_column_names = ['District', 'TotalFilings', 'Chapter7', 'Chapter11', 'Chapter12', 'Chapter13', 'Business', 'NonBusiness']
    df = df.rename(columns=dict(zip(df.columns, new_column_names)))
    df.loc[:, 'District'] = df['District'].str.replace('.', '', regex=False)
    df = df.replace(r'-$', 0, regex=True)
    df['Year'] = '20'+file[2:4]
    merged_data.append(df)

for file in xlsx_files:
    df = pd.read_excel(file)
    df = df.drop(index=df.index[:15])
    df = df.dropna(subset=df.columns[1:], how='all')
    df = df.drop(['Unnamed: 7','Unnamed: 8','Unnamed: 9', 'Unnamed: 10', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'], axis=1)
    new_column_names = ['District', 'TotalFilings', 'Chapter7', 'Chapter11', 'Chapter12', 'Chapter13', 'Business', 'NonBusiness']
    df = df.rename(columns=dict(zip(df.columns, new_column_names)))
    df.loc[:, 'District'] = df['District'].str.replace('.', '', regex=False)
    df = df.replace(r'-$', 0, regex=True)
    if file.startswith('b'):
        df['Year'] = file[11:15]
    else:
        df['Year'] = file[13:17]
    merged_data.append(df)

merged_df = pd.concat(merged_data, ignore_index=True)

WARNING *** file size (34961) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (35402) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [179]:
merged_df = merged_df.drop('Unnamed: 15',axis=1)

In [180]:
merged_df

,District,TotalFilings,Chapter7,Chapter11,Chapter12,Chapter13,Business,NonBusiness,Year
0,DC,712,358,18,0,336,36,676,2007
1,1ST,29595,16930,282,27,12355,1193,28402,2007
2,ME,2304,1819,15,6,464,152,2152,2007
3,MA,13705,9061,143,4,4496,333,13372,2007
4,NH,2983,2100,20,3,860,327,2656,2007
...,...,...,...,...,...,...,...,...,...
1702,"FL,M",26193,18829,364,6981,19,1073,25120,2019
1703,"FL,S",17940,10150,257,7513,20,629,17311,2019
1704,"GA,N",29304,14827,189,14279,9,646,28658,2019
1705,"GA,M",8100,2587,16,5471,26,151,7949,2019


In [181]:
master_path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/'
merged_df.to_csv(master_path + 'bankruptcy_filings_circuit.csv',index=False)

In [ ]:
df = df[~df['District'].str.strip().str.match(r'^\d')]
df.reset_index(drop=True,inplace=True)

In [ ]:
df

,District,TotalFilings,Chapter7,Chapter11,Chapter12,Chapter13,Business,NonBusiness
0,DC,550,297,13,0,240,27,523
1,ME,1323,1046,12,2,263,85,1238
2,MA,8400,5493,116,2,2789,253,8147
3,NH,1925,1390,12,2,521,218,1707
4,RI,1621,1288,4,0,329,48,1573
...,...,...,...,...,...,...,...,...
89,"FL,M",15756,9437,129,3,6187,521,15235
90,"FL,S",7801,5603,101,0,2096,384,7417
91,"GA,N",25035,11130,188,0,13717,899,24136
92,"GA,M",7922,2697,21,14,5190,138,7784


## 3. Health Insurance SAHIE

In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/medical/health insurance estimates SAHIE/'

In [ ]:
df = pd.read_csv(path+'sahie-2005.csv')

In [ ]:
new_column_names = df.iloc[78, :].tolist()
# Rename the columns
df = df.rename(columns=dict(zip(df.columns, new_column_names)))
df = df.drop(index=df.index[:79])
df

In [ ]:
df['year'] = 2005

In [ ]:
# Check if there is a space in the 'statefips' column
mask = df['statefips'].str.contains(' ')

# Split the values and assign them to 'statefips' and 'countyfips'
df.loc[mask, ['statefips', 'fips']] = df.loc[mask, 'statefips'].str.split(' ', 1).tolist()

# Print the modified DataFrame
print(df)

In [ ]:
# Check if there is a space in the 'statefips' column
mask = ~df['fips'].isna()
df.loc[mask, 'countyfips'] = df.loc[mask, 'fips'].astype(str) + df.loc[mask, 'countyfips'].astype(str)
df

,year,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,pop,nic,nic_moe,nui,nui_moe,pctelig,pctelig_moe,fips
79,2005,1,0,40,0,0,0,0,3934606,3322178,36389,612428,34708,15.6,0.9,NaN
80,2005,1,0,40,0,0,0,1,3934606,1019921,30814,370697,24672,9.4,0.6,NaN
81,2005,1,0,40,0,0,0,2,3934606,1314180,34518,439454,27840,11.2,0.7,NaN
82,2005,1,0,40,0,0,1,0,1936939,1615391,21605,321548,21053,16.6,1.1,NaN
83,2005,1,0,40,0,0,1,1,1936939,460268,18606,183304,15114,9.5,0.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67129,2005,56,45,50,2,0,1,0,1232,1069,77,163,57,13.2,4.6,4
67130,2005,56,45,50,2,0,1,2,1232,220,67,105,43,8.6,3.5,4
67131,2005,56,45,50,2,0,2,0,1177,995,83,182,67,15.4,5.7,4
67132,2005,56,45,50,2,0,2,2,1177,255,78,132,59,11.2,5.0,4


In [ ]:
df = df.drop(['fips'],axis=1)

In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/medical/health insurance estimates SAHIE/'
os.chdir(path)
file_pattern = "*.csv"
csv_files = glob.glob(file_pattern)

In [ ]:
csv_files[15]

'sahie-2005.csv'

In [ ]:
df = pd.read_csv(path+csv_files[15])
df['statefips'] = df['statefips'].fillna(0).astype(int)
df['statefips'] = df['statefips'].astype(int)
df = df.astype(str)
df['statefips'] = df['statefips'].astype(str).str.zfill(2)
df['countyfips'] = df['countyfips'].astype(str).str.zfill(3)
df['FIPS'] = df['statefips'].astype(str) + df['countyfips'].astype(str)
df

<ipython-input-73-196afa6c73a1>:1: DtypeWarning: Columns (2,7,8,9,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path+csv_files[15])


,year,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,pop,nic,nic_moe,nui,nui_moe,pctelig,pctelig_moe,FIPS
0,2005,01,000,40,0,0,0,0,3934606,3322178,36389,612428,34708,15.6,0.9,01000
1,2005,01,000,40,0,0,0,1,3934606,1019921,30814,370697,24672,9.4,0.6,01000
2,2005,01,000,40,0,0,0,2,3934606,1314180,34518,439454,27840,11.2,0.7,01000
3,2005,01,000,40,0,0,1,0,1936939,1615391,21605,321548,21053,16.6,1.1,01000
4,2005,01,000,40,0,0,1,1,1936939,460268,18606,183304,15114,9.5,0.8,01000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67050,2005,56,045,50,2,0,1,0,1232,1069,77,163,57,13.2,4.6,56045
67051,2005,56,045,50,2,0,1,2,1232,220,67,105,43,8.6,3.5,56045
67052,2005,56,045,50,2,0,2,0,1177,995,83,182,67,15.4,5.7,56045
67053,2005,56,045,50,2,0,2,2,1177,255,78,132,59,11.2,5.0,56045


In [ ]:
# Filter the DataFrame to keep rows where all conditions are met
df = df[(df['geocat'] == '50') & (df['agecat'] == '0') & (df['racecat'] == '0') & (df['sexcat'] == '0') & (df['iprcat'] == '0')]
df['pop'] = df['pop'].astype(int)
df['nic'] = df['nic'].astype(int)
df['pct_insured'] = df['nic'] / df['pop'] * 100
df['pct_uninsured'] = df['pctelig']

In [ ]:
df

,year,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,pop,nic,nic_moe,nui,nui_moe,pctelig,pctelig_moe,FIPS,pct_insured,pct_uninsured
145,2005,01,001,50,0,0,0,0,44780,38492,1244,6289,1070,14.0,2.4,01001,85.958017,14.0
164,2005,01,003,50,0,0,0,0,140597,111255,3911,29342,3595,20.9,2.6,01003,79.130422,20.9
183,2005,01,005,50,0,0,0,0,21930,18623,674,3307,598,15.1,2.7,01005,84.920201,15.1
202,2005,01,007,50,0,0,0,0,17731,14598,602,3133,527,17.7,3.0,01007,82.330382,17.7
221,2005,01,009,50,0,0,0,0,48775,38637,1585,10139,1448,20.8,3.0,01009,79.214762,20.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66960,2005,56,037,50,0,0,0,0,35241,29451,854,5790,727,16.4,2.1,56037,83.570273,16.4
66979,2005,56,039,50,0,0,0,0,17199,12859,577,4341,567,25.2,3.3,56039,74.765975,25.2
66998,2005,56,041,50,0,0,0,0,18589,15978,493,2612,399,14.0,2.1,56041,85.954059,14.0
67017,2005,56,043,50,0,0,0,0,6341,5353,208,988,169,15.6,2.7,56043,84.418861,15.6


In [ ]:
merged_df = pd.DataFrame()

for file in csv_files:
    if file != 'sahie-2005.csv':
        df = pd.read_csv(file, skiprows=range(1, 79))
        df = df.reset_index()
        new_column_names = df.iloc[0, :].tolist()
        # Rename the columns
        df = df.rename(columns=dict(zip(df.columns, new_column_names)))
        df = df.drop(index=df.index[:1])
        df['countyfips'] = df['countyfips'].astype(str).str.zfill(3)
        df['FIPS'] = df['statefips'].astype(str) + df['countyfips'].astype(str)
        columns_to_convert = ['geocat', 'agecat', 'racecat', 'sexcat', 'iprcat']
        df[columns_to_convert] = df[columns_to_convert].astype(str)
        # Filter the DataFrame to keep rows where all conditions are met
        df = df[(df['geocat'] == '50') & (df['agecat'] == '0') & (df['racecat'] == '0') & (df['sexcat'] == '0') & (df['iprcat'] == '0')]
        df['pct_insured'] = df['PCTLIIC']
        df['pct_uninsured'] = df['PCTELIG']
        df = df[['year','FIPS','state_name','county_name','pct_insured','pct_uninsured']]
    else:
        df = pd.read_csv(file)
        df['statefips'] = df['statefips'].fillna(0).astype(int)
        df['statefips'] = df['statefips'].astype(int)
        df = df.astype(str)
        df['statefips'] = df['statefips'].astype(str).str.zfill(2)
        df['countyfips'] = df['countyfips'].astype(str).str.zfill(3)
        df['FIPS'] = df['statefips'].astype(str) + df['countyfips'].astype(str)
        # Filter the DataFrame to keep rows where all conditions are met
        df = df[(df['geocat'] == '50') & (df['agecat'] == '0') & (df['racecat'] == '0') & (df['sexcat'] == '0') & (df['iprcat'] == '0')]
        df['pop'] = df['pop'].astype(int)
        df['nic'] = df['nic'].astype(int)
        df['pct_insured'] = df['nic'] / df['pop'] * 100
        df['pct_uninsured'] = df['pctelig']
        df = df[['year','FIPS','pct_insured','pct_uninsured']]
    merged_df = pd.concat([merged_df,df], ignore_index=True)

<ipython-input-78-8b9c79d382bf>:5: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, skiprows=range(1, 79))
<ipython-input-78-8b9c79d382bf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pct_insured'] = df['PCTLIIC']
<ipython-input-78-8b9c79d382bf>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pct_uninsured'] = df['PCTELIG']
<ipython-input-78-8b9c79d382bf>:5: Dtype

In [ ]:
merged_df

,year,FIPS,state_name,county_name,pct_insured,pct_uninsured
0,2011,01001,Alabama ...,Autauga County,86.1,13.9
1,2011,01003,Alabama ...,Baldwin County,83.4,16.6
2,2011,01005,Alabama ...,Barbour County,81.1,18.9
3,2011,01007,Alabama ...,Bibb County,84.0,16.0
4,2011,01009,Alabama ...,Blount County,81.9,18.1
...,...,...,...,...,...,...
50269,2005,56037,NaN,NaN,83.570273,16.4
50270,2005,56039,NaN,NaN,74.765975,25.2
50271,2005,56041,NaN,NaN,85.954059,14.0
50272,2005,56043,NaN,NaN,84.418861,15.6


In [ ]:
master_path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/'
merged_df.to_csv(path+'health_insurance_sahie.csv',index=False)

## 4. Opioid mortality

In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/medical/opioid mortality/NCHS_-_Drug_Poisoning_Mortality_by_County__United_States.csv'
df = pd.read_csv(path)

In [ ]:
df[['Lower Bound of Estimated Age-adjusted Death Rate', 'Upper Bound of Estimated Age-adjusted Death Rate']] = df['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'].str.split('-', expand=True)
df

,FIPS,Year,State,ST,FIPS State,County,Population,"Estimated Age-adjusted Death Rate, 16 Categories (in ranges)",Lower Bound of Estimated Age-adjusted Death Rate,Upper Bound of Estimated Age-adjusted Death Rate
0,8014,2012,Colorado,CO,8,"Broomfield County, CO",58949.0,12.1-14,12.1,14
1,41069,2013,Oregon,OR,41,"Wheeler County, OR",1390.0,14.1-16,14.1,16
2,9011,2015,Connecticut,CT,9,"New London County, CT",271863.0,18.1-20,18.1,20
3,36117,2012,New York,NY,36,"Wayne County, NY",93017.0,4.1-6,4.1,6
4,48263,2015,Texas,TX,48,"Kent County, TX",764.0,4.1-6,4.1,6
...,...,...,...,...,...,...,...,...,...,...
53382,38075,2007,North Dakota,ND,38,"Renville County, ND",2415.0,0-2,0,2
53383,31021,2000,Nebraska,NE,31,"Burt County, NE",7786.0,0-2,0,2
53384,38055,2007,North Dakota,ND,38,"McLean County, ND",8811.0,2.1-4,2.1,4
53385,53039,1999,Washington,WA,53,"Klickitat County, WA",18929.0,4.1-6,4.1,6


In [ ]:
df['FIPS'] = df['FIPS'].astype(str).str.zfill(5)
df = df[['Year','FIPS','County','Lower Bound of Estimated Age-adjusted Death Rate', 'Upper Bound of Estimated Age-adjusted Death Rate']]

In [ ]:
master_path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/'
df.to_csv(master_path+'opioid_mortality.csv',index=False)

## 5. Poverty Estimates SAIPE

In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/economic/poverty estimates'
os.chdir(path)
file_pattern = "*.xls"
xls_files = glob.glob(file_pattern)
xlsx_files = glob.glob("*.xlsx")
files = xls_files + xlsx_files

In [ ]:
### Fix format for one file
df = pd.read_excel(xls_files[0])
new_column_names = df.iloc[1, :].tolist()
# Rename the columns
df = df.rename(columns=dict(zip(df.columns, new_column_names)))
df = df.drop(index=df.index[:2])
df = df.drop(index=df.index[-3:])
df

In [ ]:
df['State FIPS'] = df['State FIPS'].astype(str).str.zfill(2)
df['County FIPS'] = df['County FIPS'].astype(str).str.zfill(3)
df['FIPS'] = df['State FIPS'].astype(str) + df['County FIPS'].astype(str)
df['Year'] = '20'+xls_files[0][3:5]
df

In [ ]:
df =df[['Year','FIPS','Name','Poverty Percent All Ages','Median Household Income']]

In [ ]:
merged_df = pd.DataFrame()

for file in xls_files:
    print(file)
    df = pd.read_excel(file)
    new_column_names = df.iloc[1, :].tolist()
    # Rename the columns
    df = df.rename(columns=dict(zip(df.columns, new_column_names)))
    df = df.drop(index=df.index[:2])
    df = df.drop(index=df.index[-3:])
    df['State FIPS'] = df['State FIPS'].astype(str).str.zfill(2)
    df['County FIPS'] = df['County FIPS'].astype(str).str.zfill(3)
    df['FIPS'] = df['State FIPS'].astype(str) + df['County FIPS'].astype(str)
    df['Year'] = '20'+file[3:5]
    df =df[['Year','FIPS','Name','Poverty Percent All Ages','Median Household Income']]
    merged_df = pd.concat([merged_df,df],axis=0)

for file in xlsx_files:
    print(file)
    df = pd.read_excel(file)
    new_column_names = df.iloc[1, :].tolist()
    # Rename the columns
    df = df.rename(columns=dict(zip(df.columns, new_column_names)))
    df = df.drop(index=df.index[:2])
    df = df.drop(index=df.index[-3:])
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.zfill(2)
    df.iloc[:, 1] = df.iloc[:, 1].astype(str).str.zfill(3)
    df['FIPS'] = df.iloc[:, 0].astype(str) + df.iloc[:, 1].astype(str)
    df['Year'] = '20'+file[3:5]
    df = df.rename(columns={'Poverty Percent, All Ages': 'Poverty Percent All Ages'})
    df =df[['Year','FIPS','Name','Poverty Percent All Ages','Median Household Income']]
    merged_df = pd.concat([merged_df,df],axis=0)

est05all.xls
est06all.xls
est07all.xls
est08all.xls
est09all.xls
est10all.xls
est11all.xls
est12all.xlsx
est13all.xlsx
est14all.xlsx
est15all.xlsx
est16all.xlsx
est17all.xlsx
est18all.xlsx
est19all.xlsx
est20all.xlsx


In [ ]:
merged_df

,Year,FIPS,Name,Poverty Percent All Ages,Median Household Income
2,2005,00000,United States,13.3,46242
3,2005,01000,Alabama,16.9,36936
4,2005,01001,Autauga County,10.4,45019
5,2005,01003,Baldwin County,11.4,42804
6,2005,01005,Barbour County,22.4,29534
...,...,...,...,...,...
3189,2020,56031,Platte County,9.9,60178
3190,2020,56033,Sheridan County,9.5,59947
3191,2020,56035,Sublette County,6.2,82442
3192,2020,56037,Sweetwater County,7.6,70583


In [ ]:
merged_df.to_csv(master_path+'poverty_estimates_saipe.csv',index=False)

## 6. Natural Disaster FEMA

In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/population/natural disaster/DisasterDeclarationsSummaries.csv'
df = pd.read_csv(path)
df

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,tribalRequest,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,lastRefresh,hash,id
0,FM-5465-NM,5465,NM,FM,2023-05-10T00:00:00.000Z,2023,Fire,LAS TUSAS FIRE,0,0,...,0,35,33,99033,Mora (County),23050,NaN,2023-05-22T03:41:22.800Z,56e2cdd7de53fd6bd0cbe22cd681d0d87708d0ac,e46df225-24a5-4178-9368-0f1c753f8748
1,FM-5465-NM,5465,NM,FM,2023-05-10T00:00:00.000Z,2023,Fire,LAS TUSAS FIRE,0,0,...,0,35,47,99047,San Miguel (County),23050,NaN,2023-05-22T03:41:22.800Z,33f747f70112aed4df749cfb1dd58f1d50f029ff,c8660535-8995-4187-af20-771b5669d4b7
2,FM-5464-RI,5464,RI,FM,2023-04-14T00:00:00.000Z,2023,Fire,QUEENS RIVER FIRE,0,0,...,0,44,9,99009,"Washington (County)(in (P)MSA 5520,6480)",23042,NaN,2023-05-22T03:41:22.800Z,1111d4d4a33f8a6e4d383ac5bd0e60b9bf142da0,9b9a9d9c-8b03-4049-9b16-a3c0d581556c
3,FM-5463-KS,5463,KS,FM,2023-04-13T00:00:00.000Z,2023,Fire,HADDAM FIRE,0,0,...,0,20,201,99201,Washington (County),23038,NaN,2023-05-22T03:41:22.800Z,78dd55a181f146622eb65835b5d794689d1ea7f2,57d936ce-a0d7-412a-9d78-18aee530c146
4,FM-5462-NE,5462,NE,FM,2023-04-09T00:00:00.000Z,2023,Fire,WACONDA-BEAVER LAKE FIRE COMPLEX,0,0,...,0,31,25,99025,Cass (County),23036,NaN,2023-05-22T03:41:22.800Z,5b35b5912f606a21ff29194317052ac7c2541be8,69325b2c-77cf-493f-8f1a-d5b5a737edf8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64390,DR-5-MT,5,MT,DR,1953-06-06T00:00:00.000Z,1953,Flood,FLOODS,0,1,...,0,30,0,0,Statewide,53006,NaN,2023-05-22T03:41:22.800Z,b3ade4aa4208956de534cfbf40e4b54fee026b04,344d65df-53e8-48bc-a77b-4fa97184dd46
64391,DR-4-MI,4,MI,DR,1953-06-02T00:00:00.000Z,1953,Tornado,TORNADO,0,1,...,0,26,0,0,Statewide,53004,NaN,2023-05-22T03:41:22.800Z,7408826e6ce51d4df16cbeb867ee887a7bdcf59f,07820d54-87a2-4189-ad23-792a89f2d3e3
64392,DR-3-LA,3,LA,DR,1953-05-29T00:00:00.000Z,1953,Flood,FLOOD,0,1,...,0,22,0,0,Statewide,53005,NaN,2023-05-22T03:41:22.800Z,dc1095d6a61af4652bbd518974069ba7cc9dcae4,44277869-a10a-4045-ba6f-8b8ca9ff803b
64393,DR-2-TX,2,TX,DR,1953-05-15T00:00:00.000Z,1953,Tornado,TORNADO & HEAVY RAINFALL,0,1,...,0,48,0,0,Statewide,53003,NaN,2023-05-22T03:41:22.800Z,f1071447976f74d3e4807ee9aeae47c03d5e0700,0c8116ba-70a4-4675-ac21-513aba0af832


In [ ]:
df['fyDeclared'] = df['fyDeclared'].astype(int)
df = df[df['fyDeclared'] >= 2000]

In [ ]:
df['fipsStateCode'] = df['fipsStateCode'].astype(str).str.zfill(2)
df['fipsCountyCode'] = df['fipsCountyCode'].astype(str).str.zfill(3)
df['FIPS'] = df['fipsStateCode'].astype(str) + df['fipsCountyCode'].astype(str)

<ipython-input-142-36014db1a9a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fipsStateCode'] = df['fipsStateCode'].astype(str).str.zfill(2)
<ipython-input-142-36014db1a9a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fipsCountyCode'] = df['fipsCountyCode'].astype(str).str.zfill(3)
<ipython-input-142-36014db1a9a1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [ ]:
count_df = df.groupby(['FIPS', 'fyDeclared'])['incidentType'].value_counts().reset_index(name='count')
count_df

,FIPS,fyDeclared,incidentType,count
0,01000,2017,Hurricane,1
1,01000,2018,Hurricane,1
2,01000,2019,Hurricane,1
3,01000,2020,Biological,4
4,01000,2020,Hurricane,2
...,...,...,...,...
32441,78030,2011,Severe Storm,1
32442,78030,2017,Hurricane,4
32443,78030,2019,Hurricane,1
32444,78030,2020,Biological,2


In [ ]:
count_df.to_csv(master_path+'natural_disaster.csv',index=False)